# FLSim Tutorial: Next char prediction with LEAF's Shakespeare


### Objectives 

By the end of this tutorial, we will have learnt how to

1. Build a data pipeline for federated learning with FLSim,
2. Create a next char prediction model compatible with FL training,
3. Create a metrics reporter to collect and report metrics,
4. Set hyperparameters for FL training, and
5. Launch an FL training flow using FLSim.

## Training a next-char predictor with FLSim

### Prerequisites
First, let us install flsim via pip with the command below:

In [1]:
# !pip install --quiet flsim

Some useful parameters for later - no need to change these.

In [2]:
USE_CUDA = True
LOCAL_BATCH_SIZE = 32
SEQ_LEN = 80
# suppress large outputs
VERBOSE = False

### 0. About the dataset

For this tutorial, we will use [LEAF's](https://leaf.cmu.edu/) Shakespeare dataset 

Before the next step in this tutorial, we need to download the dataset and partition the data by users. 
In particular, we sample 
5% of the entire dataset (`--sf 0.05`) 
in a non-IID manner (`-s niid`) 
and partition 90% of sampled users into train and 10% of sampled users 
into test (`--tf 0.90`) as opposed to splitting individual samples into train and test (`-t 'user'`) .
We require all users to have at least one sample (`-k 1`).

```
./preprocess.sh -s niid --sf 0.05 -k 1 --tf 0.90 -t 'user'

```

For more information on the various preprocessing options, see [here](https://github.com/TalwalkarLab/leaf/tree/master/data/shakespeare). You can find the LEAF paper [here](https://arxiv.org/pdf/1812.01097.pdf).


Let us now find our preprocessed data. Note that if you use different preprocessing options, you might need to change these!



In [3]:
import os
os.chdir('../../leaf/data/shakespeare')
TRAIN_DATA = os.listdir('data/train')
TRAIN_DATA = "data/train/" + TRAIN_DATA[0]

TEST_DATA = os.listdir('data/test')
TEST_DATA = "data/test/" + TEST_DATA[0]

TRAIN_DATA, TEST_DATA

('data/train/all_data_0_05_keep_1_train_9.json',
 'data/test/all_data_0_05_keep_1_test_9.json')

We can now load the training data and get a sense of how samples are distributed across users in our subset of the Sent140 dataset.


In [4]:
import json
import numpy as np

# load the training data
with open(TRAIN_DATA, "r") as f:
    training_data = json.load(f)

# how samples are distributed across users
n_samples = training_data['num_samples']
print(f"""\nNumber of samples per user:
  min={np.min(n_samples)}, 
  max={np.max(n_samples)}, 
  median={np.median(n_samples)}, 
  mean={np.mean(n_samples):.2f}, 
  std={np.std(n_samples):.2f}
  """)


Number of samples per user:
  min=30, 
  max=26131, 
  median=1595.0, 
  mean=4012.76, 
  std=5671.36
  


Let us also look at the data for an example user. Notice that there are multiple metadata fields in addition to the tweet itself and its sentiment label.

In [5]:
EXAMPLE_USER = training_data["users"][0]
print(f"Example user: {EXAMPLE_USER}")
training_data["user_data"][EXAMPLE_USER]

Example user: THE_HISTORY_OF_TROILUS_AND_CRESSIDA_PARIS


{'x': ['Else might the world convince of levity As well my undertakings as your counsels',
  'lse might the world convince of levity As well my undertakings as your counsels;',
  'se might the world convince of levity As well my undertakings as your counsels; ',
  'e might the world convince of levity As well my undertakings as your counsels; B',
  ' might the world convince of levity As well my undertakings as your counsels; Bu',
  'might the world convince of levity As well my undertakings as your counsels; But',
  'ight the world convince of levity As well my undertakings as your counsels; But ',
  'ght the world convince of levity As well my undertakings as your counsels; But I',
  'ht the world convince of levity As well my undertakings as your counsels; But I ',
  't the world convince of levity As well my undertakings as your counsels; But I a',
  ' the world convince of levity As well my undertakings as your counsels; But I at',
  'the world convince of levity As well my undert

### 1. Data pipeline

Now, let us define how to build the data pipeline for federated learning:

1. To load the training and test data, we define a new dataset class, `Sent140Dataset`, which converts each user's tweets (features) into a `torch.Tensor`, discarding tweet metadata such as the date and time of the tweet. It also stores each tweet's sentiment (label) as well.



In [6]:
import itertools
import re
import string
import unicodedata

import torch
from torch.utils.data import Dataset

ALL_LETTERS = "\n !\"&'(),-.0123456789:;>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz}"
NUM_LETTERS = len(ALL_LETTERS)

# 1. The Shakespeare dataset will store the phrases and corresponding next char for each user.

class Shakespeare(Dataset):
    def __init__(self, data_root, seq_len=80):
        self.data_root = data_root
        self.seq_len = seq_len

        with open(data_root, "r+") as f:
            self.dataset = json.load(f)

        self.data = {}
        self.targets = {}
        self.num_classes = 80  # next char prediction

        # Populate self.data and self.targets
        for user_id, user_data in self.dataset["user_data"].items():
            self.data[user_id] = self.process_x(list(user_data["x"]))
            self.targets[user_id] = self.process_y(list(user_data["y"]))

    def __len__(self):
        return len(self.data)

    def __iter__(self):
        for user_id in self.data.keys():
            yield self.__getitem__(user_id)

    def __getitem__(self, user_id: str):
        if user_id not in self.data or user_id not in self.targets:
            raise IndexError(f"User {user_id} is not in dataset")
        return self.data[user_id], self.targets[user_id]

    def process_x(self, raw_x_batch):
        x_batch = [self.word_to_indices(word) for word in raw_x_batch]
        return torch.LongTensor(x_batch)

    def process_y(self, raw_y_batch):
        y_batch = [self.letter_to_index(c) for c in raw_y_batch]
        return torch.LongTensor(y_batch)
    
    # ------------------------
    # utils for shakespeare dataset

    


    def _one_hot(self, index, size):
        '''returns one-hot vector with given size and value 1 at given index
        '''
        vec = [0 for _ in range(size)]
        vec[int(index)] = 1
        return vec
    
    def letter_to_index(self, letter):
        '''returns index of given letter in ALL_LETTERS
        '''
        return ALL_LETTERS.find(letter)


    def letter_to_vec(self, letter):
        '''returns one-hot representation of given letter
        '''
        index = ALL_LETTERS.find(letter)
        return self._one_hot(index, NUM_LETTERS)


    def word_to_indices(self, word):
        '''returns a list of character indices

        Args:
            word: string
        
        Return:
            indices: int list with length len(word)
        '''
        indices = []
        for c in word:
            indices.append(ALL_LETTERS.find(c))
        return indices

    

2. We can now load the train and test dataset using `Sent140Dataset`.


In [7]:
# 2. Load the train and test datasets.
train_dataset = Shakespeare(
    data_root=TRAIN_DATA,
    seq_len=SEQ_LEN,
)
test_dataset = Shakespeare(
    data_root=TEST_DATA,
    seq_len=SEQ_LEN,
)

In [8]:
dset = train_dataset.dataset


for user_id, user_data in dset["user_data"].items():
    print(f"User {user_id}")
    print(list(user_data["x"]))
    print(list(user_data["y"]))
    break

for user_id, user_data in dset["user_data"].items():
    print(f"User {user_id}")
    print(train_dataset.process_y(user_data["y"]))
    break

User THE_HISTORY_OF_TROILUS_AND_CRESSIDA_PARIS
['Else might the world convince of levity As well my undertakings as your counsels', 'lse might the world convince of levity As well my undertakings as your counsels;', 'se might the world convince of levity As well my undertakings as your counsels; ', 'e might the world convince of levity As well my undertakings as your counsels; B', ' might the world convince of levity As well my undertakings as your counsels; Bu', 'might the world convince of levity As well my undertakings as your counsels; But', 'ight the world convince of levity As well my undertakings as your counsels; But ', 'ght the world convince of levity As well my undertakings as your counsels; But I', 'ht the world convince of levity As well my undertakings as your counsels; But I ', 't the world convince of levity As well my undertakings as your counsels; But I a', ' the world convince of levity As well my undertakings as your counsels; But I at', 'the world convince of levit

Recall our `EXAMPLE_USER` from earlier? Their data now looks like this:

In [9]:
train_dataset[EXAMPLE_USER]

(tensor([[29, 64, 71,  ..., 57, 64, 71],
         [64, 71, 57,  ..., 64, 71, 22],
         [71, 57,  1,  ..., 71, 22,  1],
         ...,
         [70, 67, 61,  ...,  1, 60, 61],
         [67, 61, 64,  ..., 60, 61, 65],
         [61, 64, 73,  ..., 61, 65, 10]]),
 tensor([22,  1, 26,  ..., 65, 10,  1]))

To complete our data pipeline, we only need to

3. Create a data loader, which will batchify training, eval, and test data. There is no need to create a sharder since each data sample is already associated with a user. For each dataset, the data loader splits each client's data into batches of size `batch_size`. We choose not to drop the last batch.

4. Lastly, wrap the data loader with a data provider and return it. 
The data provider creates clients from the groupings in the data loader and adds metadata (e.g. number of examples, number of batches per client). 
Our data is now formatted such that the trainer will accept it.

In [10]:
from flsim.utils.example_utils import LEAFDataLoader, DataProvider

# 3. Batchify training, eval, and test data. Note that train_dataset is already sharded.
dataloader = LEAFDataLoader(
    train_dataset,
    test_dataset,
    test_dataset,
    batch_size=LOCAL_BATCH_SIZE,
    drop_last=False,
)

# 4. Wrap the data loader with a data provider.
data_provider = DataProvider(dataloader)
print(f"\nClients in total: {data_provider.num_train_users()}")

Creating FL User: 38user [00:01, 27.36user/s]
Creating FL User: 5user [00:00,  8.90user/s]
Creating FL User: 5user [00:00,  7.12user/s]


Clients in total: 38


### 2. Create the model

Now, let's see how we can create a model that is compatible with FL-training.

1. First, we define a standard, non-FL sentiment classification PyTorch `nn.Module`. In this tutorial we use a simple char-LSTM with an embedding, LSTM, and linear layer.

2. Create a `torch.device` and choose where the model will be allocated (CUDA or CPU).

As with the data pipeline, these steps are identical to creating a model in non-federated learning. Note that in contrast to non-FL learning, we haven't moved the model to device yet.

In [11]:
from torch import nn
import torch.nn.functional as F

class CharLSTM(nn.Module):
    def __init__(
        self,
        num_classes,
        n_hidden=256,
        num_embeddings=80,
        embedding_dim=8,
        seq_len=80,
        dropout_rate=0,
    ):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.n_hidden = n_hidden
        self.num_classes = num_classes
        self.seq_len = seq_len
        self.num_embeddings = num_embeddings

        self.embedding = nn.Embedding(
            num_embeddings=self.num_embeddings, embedding_dim=embedding_dim
        )
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=self.n_hidden,
            num_layers=2,
            batch_first=True,
            dropout=self.dropout_rate,
        )
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.fc = nn.Linear(self.n_hidden, self.num_classes)

    def forward(self, sentence):
        embeds = self.embedding(sentence)  
        lstm_out, _ = self.lstm(embeds)
        final_hidden_state = lstm_out[:, -1]
        return self.fc(self.dropout(final_hidden_state))

We initialize our model with such parameters that it is compatible with our dataset.

In [12]:
import torch

# 1. Define our model, a simple char-LSTM.
model = CharLSTM(
    num_classes=train_dataset.num_classes,
    n_hidden=256,
    num_embeddings=80,
    embedding_dim=8,
    seq_len=SEQ_LEN,
    dropout_rate=0.1,
)

# 2. Choose where the model will be allocated.
cuda_enabled = torch.cuda.is_available() and USE_CUDA
gpu = 2
device = torch.device("cuda:{}".format(gpu) if cuda_enabled else "cpu")

model, device

(CharLSTM(
   (embedding): Embedding(80, 8)
   (lstm): LSTM(8, 256, num_layers=2, batch_first=True, dropout=0.1)
   (dropout): Dropout(p=0.1, inplace=False)
   (fc): Linear(in_features=256, out_features=80, bias=True)
 ),
 device(type='cuda', index=2))

As with the data pipeline, there are a few extra steps that we need to take to make sure that our model is compatible with FL. In particular, we need to

3. Wrap the PyTorch module with the FLSim `FLModel`, an abstracted version of a FL-friendly model class that is accepted by the trainer and handles metric collection, as well as the forward pass for both training and evaluation. We can recover our `nn.Module` by calling `FLModel.fl_get_module()`

4. Move the model to GPU and enable CUDA if desired. `FLModel.fl_cuda()` internally calls `model.to(device)` to move the model to GPU.

In [13]:
from flsim.utils.example_utils import FLModel

# 3. Wrap the model with FLModel.
global_model = FLModel(model, device)
assert(global_model.fl_get_module() == model)

# 4. Move the model to GPU and enable CUDA if desired.
if cuda_enabled:
    global_model.fl_cuda()

### 3. Metrics Reporting

After having created our data pipeline and FL model, we will now create our metrics reporter. 
The metrics reporter allows us to collect, evaluate, and report relevant training, aggregation, and evaluation/test metrics as well as log them onto TensorBoard.



In [14]:
from flsim.interfaces.metrics_reporter import Channel
from flsim.utils.example_utils import MetricsReporter

# Create a metric reporter.
metrics_reporter = MetricsReporter([Channel.TENSORBOARD, Channel.STDOUT])

There are three functions that are of particular interest:

1. `compute_scores` computes the metrics of interest for both training and aggregation (if desired) as well as evaluation/test.

2. `create_eval_metrics` creates a dictionary that stores the value for each eval metric. 

3. `compare_metrics` compares the current eval metrics that are returned by `create_eval_metrics` to the best eval metrics so far.

For this tutorial, our only metric of interest is top-1 accuracy. In general, as with the data loading and model, you should write your own metrics reporter depending on the task.

In [15]:
import inspect

if VERBOSE:
    print(inspect.getsource(MetricsReporter.compute_scores))
    print(inspect.getsource(MetricsReporter.create_eval_metrics))
    print(inspect.getsource(MetricsReporter.compare_metrics))

### 4. Hyperparameters

We can represent the hyperparameters for FL training in a JSON config for ease of representation and we convert the JSON config to OmegaConf before passing it to the FL trainer.

In particular, we specify a FedAvg with LR implementation with 10 users per round.

In [16]:
import flsim.configs
from flsim.utils.config_utils import fl_config_from_json
from omegaconf import OmegaConf

json_config = {
    "trainer": {
        "_base_": "base_sync_trainer",
        "server": {
            "_base_": "base_sync_server",
            "server_optimizer": {
                # there are different types of server optimizers
                # fed avg with lr requires a learning rate, whereas e.g. fed_avg doesn't
                "_base_": "base_fed_avg_with_lr",
                # server's learning rate
                "lr": 1,
                # server's global momentum
                "momentum": 0.0
            },
            # aggregate client models into a single model by taking their weighted sum
            "aggregation_type": "WEIGHTED_AVERAGE",
            # type of user selection sampling
            "active_user_selector": {
                "_base_": "base_uniformly_random_active_user_selector"
            }
        },
        "client": {
            # number of client's local epochs
            "epochs": 10,
            "optimizer": {
                # client's optimizer
                "_base_": "base_optimizer_sgd",
                # client's local learning rate
                "lr": 0.8,
                # client's local momentum
                "momentum": 0
            }
        },
        # number of users per round for aggregation
        "users_per_round": 10,
        # total number of global epochs
        # total #rounds = ceil(total_users / users_per_round) * epochs
        "epochs": 20,
        # frequency of reporting train metrics
        "train_metrics_reported_per_epoch": 10,
        # keep the trained model always (as opposed to only when it
        # performs better than the previous model on eval)
        "always_keep_trained_model": False,
        # frequency of evaluation per epoch
        "eval_epoch_frequency": 1,
        "do_eval": True,
        # should we report train metrics after global aggregation
        "report_train_metrics_after_aggregation": True
    }
}

cfg = fl_config_from_json(json_config)
if VERBOSE: print(OmegaConf.to_yaml(cfg))

/home/tomas/Documents/git/FLSim/qafel/lib/python3.10/site-packages/flsim/utils/config_utils.py:198: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=None):


### 5. Training
Recall that we already built the data provider and created a model compatible with FL training. 
We also initialized a metrics reporter and set our desired hyperparameters.

Now, we only need to instantiate the trainer with the model and hyperparameter config we defined earlier to launch the FL training flow. We run FL training with the above JSON config and utilize `eval_score` to store the final evaluation metrics.



In [17]:
from hydra.utils import instantiate

# Instantiate the trainer.
trainer = instantiate(cfg.trainer, model=global_model, cuda_enabled=cuda_enabled)   

# Launch FL training.
final_model, eval_score = trainer.train(
    data_provider=data_provider,
    metrics_reporter=metrics_reporter,
    num_total_users=data_provider.num_train_users(),
    distributed_world_size=1,
)

Round:   0%|          | 0/4 [00:00<?, ?round/s]]/home/tomas/Documents/git/FLSim/qafel/lib/python3.10/site-packages/torch/nn/modules/rnn.py:911: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


After training finishes, we evaluate the model and report the accuracy on the test set before finishing this tutorial.


In [ ]:
# We can now test our trained model.
trainer.test(
    data_provider=data_provider,
    metrics_reporter=MetricsReporter([Channel.STDOUT]),
)

## Summary

In this tutorial, we first showed how to get and preprocess LEAF's Sent140 dataset. 
We then built a data provider by splitting each client's data into batches. 
We defined a simple char-LSTM as our model, wrapped it with a model compatible with FL training, and moved it to GPU. 
Lastly, we set the hyperparameters for FL training, launched the training flow, and evaluated our model.

### Additional resources

- For a more in-depth understanding of this tutorial, check out [example_utils.py](https://github.com/facebookresearch/FLSim/blob/main/flsim/utils/example_utils.py) where we define the data loader, data provider, `FLModel`, and metrics reporter that we use in this tutorial.

- [FLSim tutorials](https://github.com/facebookresearch/FLSim/tree/main/tutorials) - check out our other tutorial on image classification.

- Kairouz et al. (2021): [Advances and Open Problems in Federated Learning](https://arxiv.org/pdf/1912.04977.pdf). As the title suggests, an in-depth overview of advances and open problems in FL.

- If you're interested in federated learning with differential privacy, take a look at [Opacus](https://opacus.ai/), a library that enables training PyTorch models with differential privacy. 
You can find a blog post introducing Opacus [here](https://ai.facebook.com/blog/introducing-opacus-a-high-speed-library-for-training-pytorch-models-with-differential-privacy/).

